## Objective of Task A


A. Build a baseline model (5 marks) 

Use the Keras library to build a neural network with the following:

- One hidden layer of 10 nodes, and a ReLU activation function

- Use the adam optimizer and the mean squared error  as the loss function.

1. Randomly split the data into a training and test sets by holding 30% of the data for testing. You can use the train_test_splithelper function from Scikit-learn.

2. Train the model on the training data using 50 epochs.

3. Evaluate the model on the test data and compute the mean squared error between the predicted concrete strength and the actual concrete strength. You can use the mean_squared_error function from Scikit-learn.

4. Repeat steps 1 - 3, 50 times, i.e., create a list of 50 mean squared errors.

5. Report the mean and the standard deviation of the mean squared errors.


<a id="item31"></a>


## Download and Clean Dataset


Let's start by importing the <em>pandas</em> and the Numpy libraries.


In [1]:
import pandas as pd
import numpy as np

We will be playing around with the same dataset that we used in the videos.

<strong>The dataset is about the compressive strength of different samples of concrete based on the volumes of the different ingredients that were used to make them. Ingredients include:</strong>

<strong>1. Cement</strong>

<strong>2. Blast Furnace Slag</strong>

<strong>3. Fly Ash</strong>

<strong>4. Water</strong>

<strong>5. Superplasticizer</strong>

<strong>6. Coarse Aggregate</strong>

<strong>7. Fine Aggregate</strong>


Let's download the data and read it into a <em>pandas</em> dataframe.


In [2]:
concrete_data = pd.read_csv('https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


So the first concrete sample has 540 cubic meter of cement, 0 cubic meter of blast furnace slag, 0 cubic meter of fly ash, 162 cubic meter of water, 2.5 cubic meter of superplaticizer, 1040 cubic meter of coarse aggregate, 676 cubic meter of fine aggregate. Such a concrete mix which is 28 days old, has a compressive strength of 79.99 MPa. 


#### Let's check how many data points we have.


In [3]:
concrete_data.shape

(1030, 9)

So, there are approximately 1000 samples to train our model on. Because of the few samples, we have to be careful not to overfit the training data.


Let's check the dataset for any missing values.


In [4]:
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [5]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

The data looks very clean and is ready to be used to build our model.


The target variable in this problem is the concrete sample strength. Therefore, our predictors will be all the other columns.


In [6]:
concrete_data_columns = concrete_data.columns

predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

<a id="item2"></a>


Let's do a quick sanity check of the predictors and the target dataframes.


In [7]:
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [8]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

## Import Keras


#### Let's go ahead and import the Keras library


In [9]:
import keras

Let's import the rest of the packages from the Keras library that we will need to build our regressoin model.


In [10]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import ReLU
from keras import activations
from keras import layers

<a id='item33'></a>


## Build a Neural Network


Let's define a function that defines our regression model for us so that we can conveniently call it to create our model.


In [11]:
n_cols = predictors.shape[1]
n_cols

8

In [12]:
# define regression model
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(10, input_shape=(n_cols,)))
    model.add(layers.Activation(activations.relu))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

## Bulid pipeline for 50 random splits and models

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


In [14]:
predictors_train, predictors_test, target_train, target_test = train_test_split(predictors, target, test_size = 0.3)

Let's call the function now to create our model.


In [15]:
# build the model
model = regression_model()

Next, we will train and test the model at the same time using the _fit_ method. We will leave out 30% of the data for validation and we will train the model for 50 epochs.


In [16]:
# fit the model
MSE_list = list()
for iteration in range(50):
  predictors_train, predictors_test, target_train, target_test = train_test_split(predictors, target, test_size = 0.3)
  model.fit(predictors_train, target_train, validation_split=0.3, epochs=50, verbose=2)
  MSE = mean_squared_error(target_test, model.predict(predictors_test))
  MSE_list.append(MSE)

Strumieniowane dane wyjściowe obcięte do 5000 ostatnich wierszy.
Epoch 1/50
16/16 - 14s - loss: 291186.3438 - val_loss: 240665.8750
Epoch 2/50
16/16 - 0s - loss: 212302.5781 - val_loss: 173531.1094
Epoch 3/50
16/16 - 0s - loss: 150672.7656 - val_loss: 121242.7031
Epoch 4/50
16/16 - 0s - loss: 103292.5312 - val_loss: 81593.0859
Epoch 5/50
16/16 - 0s - loss: 68231.3438 - val_loss: 52723.2812
Epoch 6/50
16/16 - 0s - loss: 43094.6445 - val_loss: 32597.2695
Epoch 7/50
16/16 - 0s - loss: 26002.0547 - val_loss: 19172.2422
Epoch 8/50
16/16 - 0s - loss: 14999.5654 - val_loss: 10878.3193
Epoch 9/50
16/16 - 0s - loss: 8412.4209 - val_loss: 6192.5269
Epoch 10/50
16/16 - 0s - loss: 4857.8711 - val_loss: 3721.8750
Epoch 11/50
16/16 - 0s - loss: 3083.8684 - val_loss: 2551.7061
Epoch 12/50
16/16 - 0s - loss: 2277.8301 - val_loss: 2055.4150
Epoch 13/50
16/16 - 0s - loss: 1928.8344 - val_loss: 1852.6875
Epoch 14/50
16/16 - 0s - loss: 1782.2412 - val_loss: 1755.9312
Epoch 15/50
16/16 - 0s - loss: 1698.46

In [17]:
np.mean(MSE_list)

58.663725676063834

In [18]:
np.std(MSE_list)

30.48811850783292